# Battle of the Neighborhoods - opening restaurants in Greater Area London

## Introduction/Business Problem

&emsp;A prominent restaurant enterpreneur in UK that runs restaurants serving Italian, American and Asian dishes thinks of opening more restaurants in Greater London area to increase business coverage. London is known for its vivid cuisine scene and there is abundance of the restaurants and food joints satisfying all tastes. The problem is where to open and which restaurant to open? For instance, it would be unwise to open Italian restaurant in the area where there are already many Italian restaurant open, particularly if they have high rating. It will be hard to compete and may cause monetary losses. 
<br>&emsp;Another factor for decision is population/housing density - suburban district oftentimes are dominated by the houses for 1 family and can be charecterized by the low population density. Such districts are not able to support many small businesses - there will be not enough clients to achieve profite. In such areas many small businesses tend to concentrate in the malls, that attract people from the whole neighborhood. Opening restaurant in malls adjucent to subruban districts can be a viable option.

Enterpreneur turns to data scientits and asks to provide data-based answers for the <b>two main questions</b>:
1. Which Greater London areas are the best to open restaurant(s)?
2. Which cuisine(s) are the best option to open in the identified areas?

<b>Optional question</b> that we will try to find answer for is what price policy of new restaurant might be the most successful in a given neighboordhood. 


## Data sources and description

&emsp;Greater London Area that will be studying has 32 local government districts called <b>borough</b> plus special district of City of London. Each borough is divided by <b>wards</b> which serve as primary electoral geographical unit. We will use wards as a proxy for the neighborhood. We will exclude City of London from the analysis an concentrate on all other areas of inner and outer London.
<br>&emsp;For this project we will utilize following datasets:</br>
1. [Greater London Area (GLA) Land Area and Population Density, Ward and Borough](https://data.london.gov.uk/dataset/land-area-and-population-density-ward-and-borough). This open dataset is based on the census from 2011 with some updated data from 2016, as well as projection of population till year 2050. We will use predicted for 2021 data on total population, population density, area. It is clear that population density is population divided by area. At this stage of the project we can not yet make decision which parameter exactly we will be using. We want to discriminate between densely and sparsely populated areas.
2. [Postcode directory of London](https://data.london.gov.uk/dataset/postcode-directory-for-london). This data contains information on postcodes of wards and their coordinates. There are several sets of coordinates for each ward (corresponding to many postcodes). We will perform simple averaging of coordinates to find proxy for the ward center.
3. Foursquare API to fetch infomation about venues in each of the wards. We will use coordinates obtained from the dataset #2. We will be extracting information about type of the venue and potentially reviews of venues. 
4. Optionally we will be looking at the [wellbeing of neighborhood/wards](https://data.london.gov.uk/london-ward-well-being-scores/) to identify most appropriate price range of proposed restaurant. 

### Samples of data
Let's read data and inspect it a bit.


#### Starting from the data on population density per ward.

In [2]:
import pandas as pd
import numpy as np
import random
import requests

In [3]:
wards = pd.read_csv('housing-density-ward.csv')
wards.head()

,Code,Borough,Ward_Name,Year,Population,Hectares,Square_Kilometres,Population_per_hectare,Population_per_square_kilometre
0,E05000026,Barking and Dagenham,Abbey,2011,12904,127.9,1.279,100.891321,10089.132130
1,E05000027,Barking and Dagenham,Alibon,2011,10468,136.1,1.361,76.914034,7691.403380
2,E05000028,Barking and Dagenham,Becontree,2011,11638,128.4,1.284,90.638629,9063.862928
3,E05000029,Barking and Dagenham,Chadwell Heath,2011,10098,338.0,3.380,29.875740,2987.573964
4,E05000030,Barking and Dagenham,Eastbrook,2011,10581,345.4,3.454,30.634047,3063.404748


This data set contains historical data from censes 2011 as well as projections. Let's do basic cleaning of data:

1. We will drop columns that report numbers in hectares
2. Select data from 2021

In [4]:
wards.dtypes

Code                                object
Borough                             object
Ward_Name                           object
Year                                 int64
Population                           int64
Hectares                           float64
Square_Kilometres                  float64
Population_per_hectare             float64
Population_per_square_kilometre    float64
dtype: object

In [6]:
wards.drop(['Hectares', 'Population_per_hectare'], axis=1, inplace = True)
wards = wards[wards.Year == 2021]
wards.rename(columns={'Square_Kilometres': 'km2', 'Population_per_square_kilometre': 'Population_density'}, inplace=True)
wards.head()

,Code,Borough,Ward_Name,Year,Population,km2,Population_density
6240,E05000026,Barking and Dagenham,Abbey,2021,16938,1.279,13243.158720
6241,E05000027,Barking and Dagenham,Alibon,2021,11323,1.361,8319.617928
6242,E05000028,Barking and Dagenham,Becontree,2021,14891,1.284,11597.352020
6243,E05000029,Barking and Dagenham,Chadwell Heath,2021,11297,3.380,3342.307692
6244,E05000030,Barking and Dagenham,Eastbrook,2021,11032,3.454,3193.977997


#### Postcodes and coordinates data


In [8]:
postcodes = pd.read_csv('london_postcodes-ons-postcodes-directory-nov2020.csv')

In [9]:
postcodes.head()

,pcd,pcd2,pcds,dointr,doterm,oscty,ced,oslaua,osward,parish,...,ru11ind,oac11,lat,long,lep1,lep2,pfa,imd,calncv,stp
0,BR1 1AA,BR1 1AA,BR1 1AA,201605,NaN,E99999999,E99999999,E09000006,E05000109,E43000196,...,A1,4C3,51.401546,0.015415,E37000023,NaN,E23000001,20532,E56000010,E54000030
1,BR1 1AB,BR1 1AB,BR1 1AB,201203,NaN,E99999999,E99999999,E09000006,E05000109,E43000196,...,A1,2D1,51.406333,0.015208,E37000023,NaN,E23000001,10169,E56000010,E54000030
2,BR1 1AD,BR1 1AD,BR1 1AD,201409,201709.0,E99999999,E99999999,E09000006,E05000109,E43000196,...,A1,4C3,51.400057,0.016715,E37000023,NaN,E23000001,20532,E56000010,E54000030
3,BR1 1AE,BR1 1AE,BR1 1AE,200808,NaN,E99999999,E99999999,E09000006,E05000109,E43000196,...,A1,2D1,51.404543,0.014195,E37000023,NaN,E23000001,19350,E56000010,E54000030
4,BR1 1AF,BR1 1AF,BR1 1AF,201505,NaN,E99999999,E99999999,E09000006,E05000109,E43000196,...,A1,4C3,51.401392,0.014948,E37000023,NaN,E23000001,20532,E56000010,E54000030


There are only few columns that we need from this dataset: osward - code of the word, lat and long. Let's pick them. Let's also rename osward to Code to have it matching wards dataframe.

In [10]:
postcodes = postcodes[['osward','lat', 'long']]
postcodes.rename(columns={"osward": "Code"}, inplace=True)
postcodes.head()

,Code,lat,long
0,E05000109,51.401546,0.015415
1,E05000109,51.406333,0.015208
2,E05000109,51.400057,0.016715
3,E05000109,51.404543,0.014195
4,E05000109,51.401392,0.014948


#### Foursquare API
Let's take one ward, get proxy for its coordinates and run one query to get venues in that ward. I will hide my API details when I will upload this notebook to github.

In [25]:
# API details
CLIENT_ID = 'XXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXX' # your Foursquare Secret
ACCESS_TOKEN = 'XXXXX' # your FourSquare Access Token
VERSION = '20210427'
LIMIT = 100

Let's select random ward

In [13]:
n = random.randint(0,wards.shape[0]) 
print('Random ward is', n, ', ', wards.iloc[n,2])
code= wards.iloc[n,0]

Random ward is 395 ,  Ferndale


In [14]:
# Define function the calculates cenroid of points based on coordinates of postal codes
def centeroid(arr):
    length = arr.shape[0]
    sum_x = np.sum(arr[:, 1])
    sum_y = np.sum(arr[:, 0])
    return sum_x/length, sum_y/length
                   
coords = postcodes[postcodes.Code==code].iloc[:,1:3].to_numpy()
x, y = centeroid(coords) 
print("Ward's latitude is", x)
print("Wards's longitude is", y)

Ward's latitude is -0.11768396534653464
Wards's longitude is 51.466166396039604


Now we are ready to make API call

In [26]:
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            y, 
            x, 
            radius, 
            LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=XXXXX&client_secret=XXXXX&v=20210427&ll=51.466166396039604,-0.11768396534653464&radius=500&limit=100'

In [16]:
results = requests.get(url).json()["response"]['groups'][0]['items']

In [17]:
venues_list=[]
venues_list.append([(
            wards.iloc[n,0],
            wards.iloc[n,2], 
            y, 
            x, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

In [18]:
nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Code','Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

In [19]:
nearby_venues.head(10)

,Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,E05000421,Ferndale,51.466166,-0.117684,O2 Academy,51.465588,-0.114904,Music Venue
1,E05000421,Ferndale,51.466166,-0.117684,O Cantinho de Portugal,51.467132,-0.117572,Portuguese Restaurant
2,E05000421,Ferndale,51.466166,-0.117684,Upstairs At The Department Store,51.464184,-0.115566,Restaurant
3,E05000421,Ferndale,51.466166,-0.117684,London Beer Lab,51.463363,-0.116362,Brewery
4,E05000421,Ferndale,51.466166,-0.117684,The Duke of Edinburgh,51.463250,-0.120031,Pub
5,E05000421,Ferndale,51.466166,-0.117684,The Queen's Head,51.467406,-0.118023,Pub
6,E05000421,Ferndale,51.466166,-0.117684,DW Fitness First,51.464598,-0.115210,Gym / Fitness Center
7,E05000421,Ferndale,51.466166,-0.117684,Phonox,51.464447,-0.114420,Nightclub
8,E05000421,Ferndale,51.466166,-0.117684,The Craft Beer Co.,51.463385,-0.114036,Beer Bar
9,E05000421,Ferndale,51.466166,-0.117684,Pop Brixton,51.463393,-0.112368,Food Court


We have successfully made API call and got data for one ward.

#### Well-being data

In [20]:
wellbeing = pd.read_csv('well-being.csv', sep=';')

In [21]:
wellbeing.head()

,New ward code,Borough,Ward name,Thematic measure
0,E05000637,Westminster,Knightsbridge and Belgravia,17.217884
1,E05000384,R B of Kensington and Chelsea,Campden,13.741583
2,E05000127,Bromley,West Wickham,11.572839
3,E05000529,Richmond upon Thames,South Twickenham,11.566019
4,E05000517,Richmond upon Thames,East Sheen,11.435223


This dataset contain code of the ward, borough, name and wellbeing measure. Let's rename columns. We can also drop columns Borough and Ward name since all we need is Code of the ward to identify it. 

In [22]:
wellbeing.rename(columns={'New ward code': 'Code', 'Thematic measure': 'Wellbeing'}, inplace=True)
wellbeing.drop(['Borough', 'Ward name'], axis=1, inplace=True)
wellbeing.head()

,Code,Wellbeing
0,E05000637,17.217884
1,E05000384,13.741583
2,E05000127,11.572839
3,E05000529,11.566019
4,E05000517,11.435223


Much better! We have looked into all our datasets. Next steps will be data wrangling and further cleanup.